### Synapse Lakehouse Sync Tracking Table Log Success

**This notebook calls the ClosedEntry function that updates the log entry record that the sync was a sucess in the _SynapseLakehouseTracking table.**

#### Steps
1. Import the Synapse Lakehouse Sync Functions
2. Standardize the ADLS path provided
3. Instantiant the Tracker class
4. Call the CloseEntry function to update the Tracker record that the sync was successful

#### Notebook Parameters
- **PoolName** - The Synapse Dedicated Pool name that the data will be loaded into
- **SynapseLakehouseSyncKey** - A pipe delimited hash (md5) of the pool name, schema name, table name, and row insert date time
- **SynapseWorkspaceName** - The Synapse workspace name
- **SyncFolderPathFull** - The full path in either abfss or https format that the CDC changes will be loaded to as well as where the _SynapseLakehouseSync delta table is stored
- **TableRowCountSynapse** - The row count in the Synapse Dedicated Pool table

In [ ]:
SynapseLakehouseSyncParameters = '''
{
    "PoolName": "DataWarehouse",
    "SynapseWorkspaceName": "synapsesyncqbq",
    "SynapseLakehouseSyncKey": "cdae7bfac3427a0029b86bada994290d",
    "TableRowCountSynapse": 606,
    "SynapseSyncDataADLSFullPath": "abfss://gold@enterprisedatalakeqbq.dfs.core.windows.net/Sample/"
}
'''

In [0]:
import json

SynapseLakehouseSyncParameters = json.loads(SynapseLakehouseSyncParameters)

print(json.dumps(SynapseLakehouseSyncParameters, indent=4))

In [0]:
%run "/Synapse Lakehouse Sync/Synapse Lakehouse Sync Functions"

In [0]:
SyncFolderPathDict = UnifyFolderPaths('SynapseSync', f'{SynapseLakehouseSyncParameters["SynapseSyncDataADLSFullPath"].rstrip("/")}')

print(SyncFolderPathDict)

In [0]:
import time

track = Tracker(SynapseWorkspaceName=SynapseLakehouseSyncParameters['SynapseWorkspaceName'], PoolName=SynapseLakehouseSyncParameters['PoolName'], TrackerFolderPath=SyncFolderPathDict["SynapseSync_abfss"])

# Due to the multiple updates to the delta tracking table, we will attempt 3 times waiting 5 seconds between attempts
for _ in range(3):
    try:
        track.CloseEntry(SynapseLakehouseSyncParameters['SynapseLakehouseSyncKey'], SynapseLakehouseSyncParameters['TableRowCountSynapse'])
    except Exception as e:
        print(f'Error updating entry: {e}')
        time.sleep(5) #wait for 5 seconds to try again
        